#### This is my Coursera Assignement on Clustering

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

!conda install -c conda-forge beautifulsoup4 --yes # install BeautifulSoup 4

import requests # library to handle requests
print('Libraries Ready')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00  35.81 MB/s
Libraries Ready


In [2]:
# assign Wiki page to a variable
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# assign the result of a request to a variable

page = requests.get(url)


In [4]:
# import Beautiful Soup into the Python console:

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.text, 'html.parser')


In [5]:
# find the table object
table_content = soup.find(class_="wikitable sortable")

#find all the rows
tb_rows = table_content.find_all("tr")


In [6]:
data = []  

# insert the table data in a list object
for row in tb_rows:
    cols = row.find_all('td')
    
    # insert data to list if its not emty
    if len(cols) != 0:
        cols = [ele.text.strip() for ele in cols]
        
        # insert only if 2nd element of list (Borough) is not equal to Not Assigned
        if cols[1] != 'Not assigned':
            data.append([ele for ele in cols if ele])

In [7]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
My_DF = pd.DataFrame(data, columns=column_names)

# replace Not assigned Neighborhoods with Borough name
My_DF.Neighborhood.replace('Not assigned',My_DF.Borough,inplace=True)

# Combine Neighborhoods for the same Postal Code
My_DF = My_DF.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()


In [8]:
My_DF.shape 

(103, 3)

In [9]:
# Get the Geo coordinate csv data
!wget -q -O 'geo_coord.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')


Data downloaded!


In [20]:
# Load data to a data frame
Coord_DF = pd.read_csv('geo_coord.csv')
Coord_DF.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [184]:
# Combine the Geo Coordinate Dataframe to the earlier Dataframe
Merged_DF = pd.merge(left=My_DF,right=Coord_DF, left_on='PostalCode', right_on='Postal Code')
Merged_DF.head()

# Drop the Postal Code column from tje Geo Coordinate dataframe
Merged_DF = Merged_DF.drop('Postal Code', axis=1)
Merged_DF.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [34]:
# Matplotlib and associated plotting modules
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Libraries Ready')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  18.62 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  19.68 MB/s
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries Ready


### Create a new dataframe and map of the selected borough data.

In [210]:
York_DF = Merged_DF[Merged_DF['Borough'] == 'York'].reset_index(drop=True)
York_DF.head()
York_DF.shape

(5, 5)

#### Show the map for the selected Borough

In [211]:
address = 'York, Toronto'

geolocator = Nominatim()
location2 = geolocator.geocode(address)
latitude2 = location2.latitude
longitude2 = location2.longitude

# create map of Scarborough using latitude and longitude values
York_Map = folium.Map(location=[latitude2, longitude2], zoom_start=11)

# add markers to map
for lat, lng, label in zip(York_DF['Latitude'], York_DF['Longitude'], York_DF['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(York_Map)  
    
York_Map

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


#### Explore the Neiborhoods of York

In [214]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#### Explore the neighborhoods in York

In [215]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [216]:
York_Venues = getNearbyVenues(names=York_DF['Neighborhood'],
                                   latitudes=Scarborough_DF['Latitude'],
                                   longitudes=Scarborough_DF['Longitude']
                                  )

Humewood-Cedarvale
Caledonia-Fairbanks
Del Ray, Keelesdale, Mount Dennis, Silverthorn
The Junction North, Runnymede
Weston


In [218]:
# York Venues

print(York_Venues.shape)
York_Venues.head()

(19, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Humewood-Cedarvale,43.693781,-79.428191,Cedarvale Park,43.692535,-79.428705,Field
1,Humewood-Cedarvale,43.693781,-79.428191,Phil White Arena,43.691303,-79.431761,Hockey Arena
2,Humewood-Cedarvale,43.693781,-79.428191,Cedarvale Ravine,43.690188,-79.426106,Trail
3,Humewood-Cedarvale,43.693781,-79.428191,Pigeon Trodden Lawns,43.695418,-79.422929,Park
4,Caledonia-Fairbanks,43.689026,-79.453512,Shoppers Drug Mart,43.690651,-79.456310,Pharmacy


In [219]:
York_Venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(Scarborough_Venues['Venue Category'].unique())))

There are 15 uniques categories.


#### Analyze the Neighborhod

In [221]:
# one hot encoding
York_onehot = pd.get_dummies(York_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
York_onehot['Neighborhood'] = York_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [York_onehot.columns[-1]] + list(York_onehot.columns[:-1])
York_onehot = York_onehot[fixed_columns]

In [223]:
York_onehot.shape

(19, 16)

In [224]:
York_grouped = York_onehot.groupby('Neighborhood').mean().reset_index()
York_grouped.shape

(5, 16)

#### Print each neighborhood along with the top 3 most common venues

In [227]:
num_top_venues = 3

for hood in York_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[York_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caledonia-Fairbanks----
                  venue  freq
0                  Park  0.33
1  Fast Food Restaurant  0.17
2                Market  0.17


----Del Ray, Keelesdale, Mount Dennis, Silverthorn----
            venue  freq
0     Coffee Shop  0.25
1      Restaurant  0.25
2  Sandwich Place  0.25


----Humewood-Cedarvale----
          venue  freq
0         Field  0.25
1  Hockey Arena  0.25
2          Park  0.25


----The Junction North, Runnymede----
         venue  freq
0     Bus Line   0.5
1  Pizza Place   0.5
2  Coffee Shop   0.0


----Weston----
               venue  freq
0               Park  0.67
1  Convenience Store  0.33
2           Bus Line  0.00




In [229]:
# Insert in a dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [231]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = York_grouped['Neighborhood']

for ind in np.arange(York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Caledonia-Fairbanks,Park,Women's Store,Pharmacy,Market,Fast Food Restaurant,Turkish Restaurant,Trail,Sandwich Place,Restaurant,Pizza Place
1,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",Turkish Restaurant,Sandwich Place,Restaurant,Coffee Shop,Women's Store,Trail,Pizza Place,Pharmacy,Park,Market
2,Humewood-Cedarvale,Trail,Park,Hockey Arena,Field,Women's Store,Turkish Restaurant,Sandwich Place,Restaurant,Pizza Place,Pharmacy
3,"The Junction North, Runnymede",Pizza Place,Bus Line,Women's Store,Turkish Restaurant,Trail,Sandwich Place,Restaurant,Pharmacy,Park,Market
4,Weston,Park,Convenience Store,Women's Store,Turkish Restaurant,Trail,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Market


### Cluster the Neighborhoods

In [232]:

# import k-means from clustering stage
from sklearn.cluster import KMeans

kclusters = 3

York_grouped_clustering = York_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 0, 2, 0], dtype=int32)

In [233]:
York_DF.head()
York_DF.shape

York_merged = York_DF

York_merged = York_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

York_merged['Cluster Labels'] = kmeans.labels_


### Show map of Clusters

In [234]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(York_merged['Latitude'], York_merged['Longitude'], York_merged['Neighborhood'], York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analysis of Clusters

#### Cluster 1

In [207]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,York,Trail,Park,Hockey Arena,Field,Women's Store,Turkish Restaurant,Sandwich Place,Restaurant,Pizza Place,Pharmacy,0
2,York,Turkish Restaurant,Sandwich Place,Restaurant,Coffee Shop,Women's Store,Trail,Pizza Place,Pharmacy,Park,Market,0
4,York,Park,Convenience Store,Women's Store,Turkish Restaurant,Trail,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Market,0


### Cluster 2

In [208]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
1,York,Park,Women's Store,Pharmacy,Market,Fast Food Restaurant,Turkish Restaurant,Trail,Sandwich Place,Restaurant,Pizza Place,1


### Cluster 3

In [209]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
3,York,Pizza Place,Bus Line,Women's Store,Turkish Restaurant,Trail,Sandwich Place,Restaurant,Pharmacy,Park,Market,2
